# Python version
Python 3.13.0 (tags/v3.13.0:60403a5, Oct  7 2024, 09:38:07) [MSC v.1941 64 bit (AMD64)] on win32

# Overleaf document
https://www.overleaf.com/read/ggqqbccgrqbg#eff08f

# Household data
Using dataset "household_data_60min_singleindex.csv" from https://data.open-power-system-data.org/household_data/ and numpy, we can create the following household data:
| Household dataframe | `timestamp` | `D_max` | `D_min` | `g_pv` | `alpha` |
|---------------------|-------------|---------|---------|--------|---------|
| df_1 | cet_cest_timestamp | DE_KN_residential1_dishwasher<br>+DE_KN_residential1_freezer<br>+DE_KN_residential1_heat_pump<br>+DE_KN_residential1_washing_machine | DE_KN_residential1_freezer | DE_KN_residential1_pv | $random([1.5, 3.5])$ |
| df_2 | cet_cest_timestamp | DE_KN_residential3_circulation_pump<br>+DE_KN_residential3_dishwasher<br>+DE_KN_residential3_freezer<br>+DE_KN_residential3_refrigerator<br>+DE_KN_residential3_washing_machine | DE_KN_residential3_freezer<br>+DE_KN_residential3_refrigerator | DE_KN_residential3_pv | $random([1.5, 3.5])$ |
| df_3 | cet_cest_timestamp | DE_KN_residential4_dishwasher<br>+DE_KN_residential4_ev<br>+DE_KN_residential4_freezer<br>+DE_KN_residential4_heat_pump<br>+DE_KN_residential4_refrigerator<br>+DE_KN_residential4_washing_machine | DE_KN_residential4_refrigerator<br>+DE_KN_residential4_freezer | DE_KN_residential4_pv | $random([1.5, 3.5])$ |
| df_4 | cet_cest_timestamp | DE_KN_residential6_circulation_pump<br>+DE_KN_residential6_dishwasher<br>+DE_KN_residential6_freezer<br>+DE_KN_residential6_washing_machine | DE_KN_residential6_freezer | DE_KN_residential6_pv | $random([1.5, 3.5])$ |

`beta` for each household is a constant value and `H_l` is updated for each household after each optimization. We can generate `beta` as a random value in $[0.5,0.7]$ for each household and store in a dictionary. `H_l` can be computed for each time slot and stored in the same dictionary.

In [59]:
import pandas as pd
import numpy as np

In [91]:
# these are the columns that we want to use
relevant_cols = ['cet_cest_timestamp', 'DE_KN_residential1_dishwasher', 'DE_KN_residential1_freezer',
                 'DE_KN_residential1_heat_pump', 'DE_KN_residential1_pv', 'DE_KN_residential1_washing_machine',
                 'DE_KN_residential3_circulation_pump', 'DE_KN_residential3_dishwasher', 'DE_KN_residential3_freezer',
                 'DE_KN_residential3_pv', 'DE_KN_residential3_refrigerator', 'DE_KN_residential3_washing_machine',
                 'DE_KN_residential4_dishwasher', 'DE_KN_residential4_ev', 'DE_KN_residential4_freezer',
                 'DE_KN_residential4_heat_pump', 'DE_KN_residential4_pv', 'DE_KN_residential4_refrigerator',
                 'DE_KN_residential4_washing_machine', 'DE_KN_residential6_circulation_pump', 'DE_KN_residential6_dishwasher',
                 'DE_KN_residential6_freezer', 'DE_KN_residential6_pv', 'DE_KN_residential6_washing_machine'
                ]

# extract the columns we want to use and drop any rows containing NaN cells
raw_household_data = pd.read_csv('household_data_60min_singleindex.csv', usecols=relevant_cols).dropna()

In [156]:
# create a dataframe for each household
# shorten timestamp header
# rename appliance headers to remove "DE_KN_residentialx_"
# rename "DE_KN_residentialx_pv" to "g_pv"
df_1 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential1_dishwasher',
                           'DE_KN_residential1_freezer',
                           'DE_KN_residential1_heat_pump',
                           'DE_KN_residential1_pv',
                           'DE_KN_residential1_washing_machine'
                          ]].copy()
df_1 = df_1.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential1_dishwasher': 'dishwasher',
                            'DE_KN_residential1_freezer': 'freezer',
                            'DE_KN_residential1_heat_pump': 'heat_pump',
                            'DE_KN_residential1_pv': 'g_pv',
                            'DE_KN_residential1_washing_machine': 'washing_machine'
                            })
df_2 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential3_circulation_pump',
                           'DE_KN_residential3_dishwasher',
                           'DE_KN_residential3_freezer',
                           'DE_KN_residential3_pv',
                           'DE_KN_residential3_refrigerator',
                           'DE_KN_residential3_washing_machine'
                           ]].copy()
df_2 = df_2.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential3_circulation_pump': 'circulation_pump',
                            'DE_KN_residential3_dishwasher': 'dishwasher',
                            'DE_KN_residential3_freezer': 'freezer',
                            'DE_KN_residential3_pv': 'g_pv',
                            'DE_KN_residential3_refrigerator': 'refrigerator',
                            'DE_KN_residential3_washing_machine': 'washing_machine'
                            })
df_3 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential4_dishwasher',
                           'DE_KN_residential4_ev',
                           'DE_KN_residential4_freezer',
                           'DE_KN_residential4_heat_pump',
                           'DE_KN_residential4_pv',
                           'DE_KN_residential4_refrigerator',
                           'DE_KN_residential4_washing_machine'
                           ]].copy()
df_3 = df_3.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential4_dishwasher': 'dishwasher',
                            'DE_KN_residential4_ev': 'ev',
                            'DE_KN_residential4_freezer': 'freezer',
                            'DE_KN_residential4_heat_pump': 'heat_pump',
                            'DE_KN_residential4_pv': 'g_pv',
                            'DE_KN_residential4_refrigerator': 'refrigerator',
                            'DE_KN_residential4_washing_machine': 'washing_machine'
                            })
df_4 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential6_circulation_pump',
                           'DE_KN_residential6_dishwasher',
                           'DE_KN_residential6_freezer',
                           'DE_KN_residential6_pv',
                           'DE_KN_residential6_washing_machine'
                           ]].copy()
df_4 = df_4.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential6_circulation_pump': 'circulation_pump',
                            'DE_KN_residential6_dishwasher': 'dishwasher',
                            'DE_KN_residential6_freezer': 'freezer',
                            'DE_KN_residential6_pv': 'g_pv',
                            'DE_KN_residential6_washing_machine': 'washing_machine'
                            })

In [157]:
# compute D_max and D_min for each household
# D_max is sum of all appliances
# D_min is sum of refrigerator and freezer
def compute_Ds(dataframe):
    D_max_cols = []
    D_min_cols = []
    for label in dataframe.columns:
        if label not in ['timestamp', 'g_pv']:
            D_max_cols.append(label)
            if label in ['freezer', 'refrigerator']:
                D_min_cols.append(label)
    dataframe['D_max'] = dataframe[D_max_cols].sum(axis=1)
    dataframe['D_min'] = dataframe[D_min_cols].sum(axis=1)
    return dataframe
df_1 = compute_Ds(df_1)
df_2 = compute_Ds(df_2)
df_3 = compute_Ds(df_3)
df_4 = compute_Ds(df_4)

In [158]:
# delete appliance columns
df_1 = df_1.drop(columns=['dishwasher', 'freezer', 'heat_pump', 'washing_machine'])
df_2 = df_2.drop(columns=['circulation_pump', 'dishwasher', 'freezer', 'refrigerator', 'washing_machine'])
df_3 = df_3.drop(columns=['dishwasher', 'ev', 'freezer', 'heat_pump', 'refrigerator', 'washing_machine'])
df_4 = df_4.drop(columns=['circulation_pump', 'dishwasher', 'freezer', 'washing_machine'])

In [160]:
# generate alpha columns
df_1['alpha'] = np.random.default_rng(10).integers(low=15, high=35, size=len(df_1), endpoint=True).astype('f') / 10
df_2['alpha'] = np.random.default_rng(20).integers(low=15, high=35, size=len(df_2), endpoint=True).astype('f') / 10
df_3['alpha'] = np.random.default_rng(30).integers(low=15, high=35, size=len(df_3), endpoint=True).astype('f') / 10
df_4['alpha'] = np.random.default_rng(40).integers(low=15, high=35, size=len(df_4), endpoint=True).astype('f') / 10

In [218]:
# create household class
class Household:
    beta = None # constant in (0,1]
    H_l = 0 # updated every time slot
    data = None # dataframe containing the household's data
    has_surplus = True # flag for whether the household has energy surplus or deficit for current time slot
    # parameters used in optimization iterations
    params = {'V': None,
              'k': 1,
              'xi_ch': 1,
              'xi_dis': 1,
             }
    # need to figure out how to do
    #def update_state(timestamp):
    #    rec = data.loc[(data['timestamp'] == timestamp)]
    #    if (rec['g_pv'].item()-rec['D_max'].item()) >= 0:
    #        has_surplus = True
    #    else:
    #        has_surplus = False
    #def update_H():
    #    H_l = 0 # actually calculate H_l(t+1) here

# create household objects
household_1 = Household
household_1.data = df_1
household_2 = Household
household_2.data = df_2
household_3 = Household
household_3.data = df_3
household_4 = Household
household_4.data = df_4

In [219]:
# create beta values dictionary where betas are (reproducibly) randomly generated in [0.5,0.7]
rng = np.random.default_rng(seed=42)
generated_betas = rng.integers(low=50, high=70, size=4, endpoint=True) / 100
household_1.beta = generated_betas[0].item()
household_2.beta = generated_betas[1].item()
household_3.beta = generated_betas[2].item()
household_4.beta = generated_betas[3].item()

In [220]:
household_1.beta

0.59

# Main grid pricing data
Use BC Hydro's residential time-of-day pricing scheme (https://app.bchydro.com/accounts-billing/rates-energy-use/electricity-rates/residential-rates/tiered-time-of-day.html), forgoing the daily surcharge for simplicity:
| Description | Time | \$/kWh  |
|-------------|------|---------|
| Off-peak    | $[07:00, 16:00) \cup [21:00, 23:00)$ | 0.1097 |
| On-peak     | $[16:00, 21:00)$                     | 0.1597 |
| Overnight   | $[23:00, 07:00)$                     | 0.0597 |

The unit price of main grid energy can be generated for a given timestamp using a function.

In [114]:
from datetime import datetime

In [168]:
def price(current_timestamp):
    # current_timestamp is a string with the following format
    # YYYY-MM-DDTHH:MM:SS+ZZZZ
    # e.g. 2014-12-11T18:00:00+0100
    ts = datetime.strptime(current_timestamp, '%Y-%m-%dT%H:%M:%S%z')

    # off-peak: [7,16)
    if ((ts.hour >= 7) and (ts.hour < 16)) or ((ts.hour >= 21) and (ts.hour < 23)):
        return 0.1097
    # on-peak: [16,21)
    elif (ts.hour >= 16) and (ts.hour < 21):
        return 0.1597
    # overnight
    else:
        return 0.0597

In [170]:
test_timestamps = ['2014-12-11T09:00:00+0100','2014-12-11T22:00:00+0100','2014-12-11T18:00:00+0100','2014-12-11T00:00:00+0100']
for i in test_timestamps:
    print(f'price at {i} is {price(i)} $/kWh')

price at 2014-12-11T09:00:00+0100 is 0.1097 $/kWh
price at 2014-12-11T22:00:00+0100 is 0.1097 $/kWh
price at 2014-12-11T18:00:00+0100 is 0.1597 $/kWh
price at 2014-12-11T00:00:00+0100 is 0.0597 $/kWh
